In [3]:
from great_tables import GT, md, loc, style # tabla GT / formatos
import gt_extras as gte # extras GT (resaltados)
import janitor.polars
import polars.selectors as cs
from selenium import webdriver # exportar PNG (GT)
from PIL import Image, ImageOps  # margen PNG (post)
from datetime import datetime, timedelta
%run euroligue.ipynb
%run eurocup.ipynb
%run fiba_championsleague.ipynb
%run fiba_europecup.ipynb


# Rutas de salida
OUTPUT_DIR = "../png"
OUTPUT_FILE = "desarrollo_europeo.png"

# Crear directorio png si no existe
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
hoy = datetime.today()
lunes = hoy - timedelta(days=hoy.weekday())
viernes = lunes + timedelta(days=4)
fecha_min = lunes.strftime("%d/%m")
fecha_max = viernes.strftime("%d/%m")

In [5]:
# caption-----------

twitter = (
    "<span style='color:#000000;font-family: \"Font Awesome 6 Brands\"'>&#xE61A;</span>"
)
tweetelcheff = "<span style='font-weight:bold;'>*@elcheff*</span>"
insta = (
    "<span style='color:#E1306C;font-family: \"Font Awesome 6 Brands\"'>&#xE055;</span>"
)
instaelcheff = "<span style='font-weight:bold;'>*@sport_iv0*</span>"
github = (
    "<span style='color:#000000;font-family: \"Font Awesome 6 Brands\"'>&#xF092;</span>"
)
githubelcheff = "<span style='font-weight:bold;'>*IvoVillanueva*</span>"
caption = f"**Datos**: *@EuroLeague @EuroCup* • **Gráfico**: *Ivo Villanueva* • {twitter} {tweetelcheff} • {insta} {instaelcheff} • {github} {githubelcheff}"

# === Funciones para logo cuadrado y nombre de equipo y liga ===


def add_photo_frame(
    color: str, logo: str, container_w: str = "100%", container_h: str = "100%"
) -> pl.Expr:
    # devuelve una Expr de Polars con el HTML final por fila
    return pl.format(
        "<div style='display:flex;align-items:center;justify-content:center;width:{};height:{};'>"
        "<img style='width:86px;height:86px;border-radius:20%;background-color:{};border:2px solid black;'"
        " src='{}' alt=''/></div>",
        pl.lit(container_w),
        pl.lit(container_h),
        pl.col(color),
        pl.col(logo),
    )


def label_html(jugador: str, equipo: str) -> pl.Expr:
    return pl.format(
        "<div style='line-height:1.2;margin-bottom:-4px;text-align:left;'>"
        "<span style='font-size:34px;font-weight:700;color:#000000;'>{}</span><br>"
        "<span style='font-size:26px;font-weight:400;color:#737373;'>{}</span>"
        "</div>",
        pl.col(jugador),
        pl.col(equipo),
    )


def theme_savant(table):
    return table.tab_options(
        # Tipografía base
        table_font_names="Oswald",
        table_font_size="38px",
        column_labels_font_size="28px",
        column_labels_font_weight="bold",
        # Header de columnas (Savant puro)
        column_labels_border_top_style="solid",
        column_labels_border_top_color="black",
        column_labels_border_top_width="3px",
        column_labels_border_bottom_color="white",
        # Body
        table_body_hlines_color="transparent",
        # Sin bordes exteriores
        table_border_top_style="none",
        table_border_bottom_style="none",
        table_body_border_bottom_style="none",
        table_body_border_bottom_color="white",
        # Header general
        heading_align="center",
        heading_border_bottom_style="none",
        # Row groups (limitado en Python)
        row_group_border_top_style="none",
        # Source notes
        source_notes_font_size="18px",
    )


# === lo mapeamos ===

tbl_df = (
    pl.concat([df_partidos_euroleague, df_partidos_eurocup])
    .sort("val", descending=True)
    .head(15)
    .with_columns(
        combo=label_html("player", "equipo"),
        combo_img=add_photo_frame("color", "logo"),
    )
    .select(
        "combo_img",
        "combo",
        "min",
        "pts",
        "reb",
        "ast",
        "bp",
        "br",
        "tap",
        "fa",
        "mm",
        "val",
    )
)

tbl_df

combo_img,combo,min,pts,reb,ast,bp,br,tap,fa,mm,val
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",22.0,17.0,3.0,6.0,1.0,3.0,0.0,1.0,20.0,22.0
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",26.0,20.0,7.0,2.0,1.0,0.0,0.0,1.0,-26.0,22.0
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",28.0,25.0,1.0,0.0,0.0,1.0,0.0,2.0,-6.0,21.0
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",19.0,17.0,4.0,1.0,1.0,0.0,0.0,0.0,8.0,21.0
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",19.0,14.0,3.0,0.0,1.0,2.0,0.0,0.0,15.0,18.0
…,…,…,…,…,…,…,…,…,…,…,…
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",25.0,6.0,6.0,1.0,1.0,1.0,4.0,3.0,7.0,16.0
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",17.0,12.0,2.0,1.0,1.0,0.0,0.0,1.0,-15.0,15.0
"""<div style='display:flex;align…","""<div style='line-height:1.2;ma…",14.0,5.0,7.0,1.0,0.0,2.0,0.0,0.0,3.0,15.0


In [ ]:
table = theme_savant(
    GT(tbl_df)
    .cols_label(
        combo_img="",
        combo="",
        min="MIN",
        pts="PTS",
        reb="REB",
        ast="AST",
        br="BR",
        bp="BP",
        tap="TAP",
        fa="FA",
        mm="+/-",
        val="VAL",
    )
    .cols_width(
        combo_img="120px",
        combo="400px",
        min="90px",
        pts="90px",
        reb="90px",
        ast="90px",
        br="90px",
        bp="90px",
        tap="90px",
        fa="90px",
        mm="90px",
        val="90px",
    )
    .fmt_number(columns=["min", "val"], decimals=0)
    .opt_row_striping(row_striping=True)
    .tab_header(
        title=md(
            f"<div style='display: flex; align-items:center; gap:12px;'>"
            "<img src='https://raw.githubusercontent.com/IvoVillanueva/SUPERMANAGER-BROKER-GENERAL/refs/heads/main/png/LogoSMNavidadAzulmarino.png' style='width:144px; height:144px;'/>"
            "<div style='display:flex; flex-direction: column; justify-content: center;'>"
            "<div style='font-weight:600; font-size:68px; line-height:1.2; text-align:left;'>Desarrollo Europeo</div>"
            "<div style='font-weight:400; font-size:35px; color:#8C8C8C; line-height:1.2; text-align:left;'>"
            f"Medias Desde el {fecha_min} al {fecha_max}"  # ← Sin f aquí
            "</div></div></div>"
        )
    )
    .tab_source_note(md(caption))
)
table.save(f"{OUTPUT_DIR}/{OUTPUT_FILE}")

img = Image.open(f"{OUTPUT_DIR}/{OUTPUT_FILE}")
img_out = ImageOps.expand(img, border=100, fill="white")
img_out.save(f"{OUTPUT_DIR}/desarrollo_europeo_margen.png")